In [1]:
## Some useful package imports
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np

import nltk
from nltk.stem.porter import *
import string
from nltk.tokenize import TweetTokenizer
token = TweetTokenizer()
stemmer = PorterStemmer() 
from nltk.corpus import stopwords # Import the stop word list
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the first book
data = pd.read_csv('Paula_Hawkins-The-Girl-On-The-Train.csv', sep = '\t', header = None)
# Rename columns
data.columns = ['rating', 'tail', 'title', 'review']
data['book'] = 'Paula_Hawkins-The-Girl-On-The-Train'

# Load the second book
data1 = pd.read_csv('Andy-Weir-The-Martian.csv', sep = '\t', header = None)
# Rename columns
data1.columns = ['rating', 'tail', 'title', 'review']
data1['book'] = 'Andy-Weir-The-Martian'

# Concatenate into one dataframe
data = pd.concat((data, data1))
# Reset index and remove the index column
data = data.reset_index().ix[:, 1:]


############## Concatenate the other books ########################

data1 = pd.read_csv('Donna-Tartt-The-Goldfinch.csv', sep = '\t', header = None)
data1.columns = ['rating', 'tail', 'title', 'review']
data1['book'] = 'Donna-Tartt-The-Goldfinch'

data = pd.concat((data, data1))
data = data.reset_index().ix[:, 1:]

data1 = pd.read_csv('EL-James-Fifty-Shades-of-Grey.csv', sep = '\t', header = None)
data1.columns = ['rating', 'tail', 'title', 'review']
data1['book'] = 'EL-James-Fifty-Shades-of-Grey'

data = pd.concat((data, data1))
data = data.reset_index().ix[:, 1:]

data1 = pd.read_csv('Fillian_Flynn-Gone_Girl.csv', sep = '\t', header = None)
data1.columns = ['rating', 'tail', 'title', 'review']
data1['book'] = 'Fillian_Flynn-Gone_Girl'

data = pd.concat((data, data1))
data = data.reset_index().ix[:, 1:]

data1 = pd.read_csv('John-Green-The-Fault-in-our-Stars.csv', sep = '\t', header = None)
data1.columns = ['rating', 'tail', 'title', 'review']
data1['book'] = 'John-Green-The-Fault-in-our-Stars'

data = pd.concat((data, data1))
data = data.reset_index().ix[:, 1:]

data1 = pd.read_csv('Laura-Hillenbrand-Unbroken.csv', sep = '\t', header = None)
data1.columns = ['rating', 'tail', 'title', 'review']
data1['book'] = 'Laura-Hillenbrand-Unbroken'

##########" Final concatenation ##############"
data = pd.concat((data, data1))
data = data.reset_index().ix[:, 1:]

print('The dataframe contains ', data.shape[0], ' rows, which consist of reviews from ', len(data.book.unique()), 
      ' different books\n')

for book in data.book.unique():
    print(book, ' has ', len(data[data.book == book]), ' reviews')
    
    
def process(x):
    '''
    Function to clear the html tags, then remove non-letters characters, 
    lowercase, remove stopwords, then join the resulting as a full text
    '''

    # Clean html tags
    review = BeautifulSoup(x, "html.parser").get_text() 
    
    # Remove non letters     
    letters = re.sub("[^a-zA-Z]", " ", review) 
    
    # Lowercase and split to words
    words = letters.lower().split()                             
    
    # It appears that searching in a set in Python is much more efficient than searching in a list
    stops = set(stopwords.words("english"))                  
    
    # Remove stopwords
    text = [w for w in words if not w in stops]   
    
    # Join to produce the cleaned texts 

    return( " ".join( text ))

data['processed'] = data['review'].apply(lambda x: process(x))

The dataframe contains  219242  rows, which consist of reviews from  7  different books

Paula_Hawkins-The-Girl-On-The-Train  has  37139  reviews
Andy-Weir-The-Martian  has  22571  reviews
Donna-Tartt-The-Goldfinch  has  22861  reviews
EL-James-Fifty-Shades-of-Grey  has  32977  reviews
Fillian_Flynn-Gone_Girl  has  41974  reviews
John-Green-The-Fault-in-our-Stars  has  35844  reviews
Laura-Hillenbrand-Unbroken  has  25876  reviews


In [3]:
data.processed[0]

'rachel woman considers worthless feels women valued two things looks role mother barren rather plain looking unbeknownst landlady lost job continues ride commuter train twice day unfortunately must pass home ex husband tom new wife anna recently child something rachel unable produce married moved new family home rachel shared tom posted picture newborn facebook caption never happier rachel despondency taken drinking point blackouts forgets drunk calls husband many times night even shows home signal malfunction often finds rail car stopped tracks next former home starts notice another couple live doors refers golden couple manufactures narrative lives observes day gradually become important megan golden couple disappears rachel finds integral character police investigation seen stalking neighborhood night disappearance wounds body explained megan anna look enough alike police feel may mistaken identity involved book told three voices rachel megan anna fact rachel history drunken blacko

In [21]:
def process(x, remove_stopwords=False):
    '''
    Function to clear the html tags, then remove non-letters characters, 
    lowercase, remove stopwords, then join the resulting as a full text
    '''
    # Clean html tags
    review = BeautifulSoup(x, "html.parser").get_text() 
    # Remove non letters     
    letters = re.sub("[^a-zA-Z]", " ", review) 
    # Lowercase and split to words
    words = letters.lower().split()                             
    # It appears that searching in a set in Python is much more efficient than searching in a list
    stops = set(stopwords.words("english"))                  
    # Optionally remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return words 

import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Function that splits reviews into sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False ):
    '''Function to split a review into parsed sentences. Returns a 
    list of sentences, where each sentence is a list of words
    '''
    
    # Split into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(process(raw_sentence, remove_stopwords))
            
    # Return the list of sentences (each sentence is a list of words, so this returns a list of lists
    return sentences

In [22]:
# Apply the function to the whole dataset

start = time.time()

sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)
      
print(time.time() - start)

Parsing sentences from training set


C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.. .'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\P

547.5561151504517
